In [551]:
import psycopg2
import io
import pandas as pd

In [552]:
def create_database():
    # connect to default database
    # conn = psycopg2.connect("host= localhost dbname = studentdb user = postgres password = 0507 ")
    conn = psycopg2.connect("host= localhost dbname = postgres user = postgres password = 0507 ")
    conn.set_session(autocommit = True)
    cur = conn.cursor()
    
    #create sparkify database with UTFB encoding
    cur.execute("DROP DATABASE IF EXISTS songs")
    cur.execute("CREATE DATABASE songs")
    
    #close cinnection default
    conn.close()
    
    #connecty sprkify database 
    conn = psycopg2.connect("host= localhost dbname = songs1 user=postgres password = 0507")
    cur = conn.cursor()
    
    return cur, conn    

In [553]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()
        

In [554]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()
    

In [555]:
Songs_spotify  = pd.read_csv("spotify.csv")

In [556]:
Songs_spotify.head()

,Artist,Track,Album,Album_type,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,...,Title,Channel,Views,Likes,Comments,Licensed,official_video,Stream,EnergyLiveness,most_playedon
0,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,0.705,-6.679,0.1770,0.008360,0.002330,...,Gorillaz - Feel Good Inc. (Official Video),Gorillaz,693555221.0,6220896.0,169907.0,True,True,1.040235e+09,1.150082,Spotify
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,0.703,-5.815,0.0302,0.086900,0.000687,...,Gorillaz - Rhinestone Eyes [Storyboard Film] (...,Gorillaz,72011645.0,1079128.0,31003.0,True,True,3.100837e+08,15.183585,Spotify
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,0.923,-3.930,0.0522,0.042500,0.046900,...,Gorillaz - New Gold ft. Tame Impala & Bootie B...,Gorillaz,8435055.0,282142.0,7399.0,True,True,6.306347e+07,7.956897,Spotify
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,0.739,-5.810,0.0260,0.000015,0.509000,...,Gorillaz - On Melancholy Hill (Official Video),Gorillaz,211754952.0,1788577.0,55229.0,True,True,4.346636e+08,11.546875,Spotify
4,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,0.694,-8.627,0.1710,0.025300,0.000000,...,Gorillaz - Clint Eastwood (Official Video),Gorillaz,618480958.0,6197318.0,155930.0,True,True,6.172597e+08,9.942693,Youtube


In [557]:
Songs_spotify.columns

Index(['Artist', 'Track', 'Album', 'Album_type', 'Danceability', 'Energy',
       'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Liveness', 'Valence', 'Tempo', 'Duration_min', 'Title', 'Channel',
       'Views', 'Likes', 'Comments', 'Licensed', 'official_video', 'Stream',
       'EnergyLiveness', 'most_playedon'],
      dtype='object')

In [558]:
S1_Spotify = Songs_spotify[['Artist', 'Track', 'Album', 'Album_type', 'Danceability','Views', 'Likes', 'Comments', 'Licensed']]

In [559]:
S1_Spotify.head()

,Artist,Track,Album,Album_type,Danceability,Views,Likes,Comments,Licensed
0,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,693555221.0,6220896.0,169907.0,True
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,72011645.0,1079128.0,31003.0,True
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,8435055.0,282142.0,7399.0,True
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,211754952.0,1788577.0,55229.0,True
4,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,618480958.0,6197318.0,155930.0,True


In [560]:
S1_Spotify = S1_Spotify.drop(['Licensed'],axis =1)

In [561]:
S1_Spotify.head()

,Artist,Track,Album,Album_type,Danceability,Views,Likes,Comments
0,Gorillaz,Feel Good Inc.,Demon Days,album,0.818,693555221.0,6220896.0,169907.0
1,Gorillaz,Rhinestone Eyes,Plastic Beach,album,0.676,72011645.0,1079128.0,31003.0
2,Gorillaz,New Gold (feat. Tame Impala and Bootie Brown),New Gold (feat. Tame Impala and Bootie Brown),single,0.695,8435055.0,282142.0,7399.0
3,Gorillaz,On Melancholy Hill,Plastic Beach,album,0.689,211754952.0,1788577.0,55229.0
4,Gorillaz,Clint Eastwood,Gorillaz,album,0.663,618480958.0,6197318.0,155930.0


In [562]:
S2_Spotify = Songs_spotify[['Artist', 'Danceability', 'Energy',
       'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness']]

In [563]:
S2_Spotify.head()

,Artist,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness
0,Gorillaz,0.818,0.705,-6.679,0.1770,0.008360,0.002330
1,Gorillaz,0.676,0.703,-5.815,0.0302,0.086900,0.000687
2,Gorillaz,0.695,0.923,-3.930,0.0522,0.042500,0.046900
3,Gorillaz,0.689,0.739,-5.810,0.0260,0.000015,0.509000
4,Gorillaz,0.663,0.694,-8.627,0.1710,0.025300,0.000000


In [564]:
cur, conn = create_database()

ObjectInUse: database "songs" is being accessed by other users
DETAIL:  There are 2 other sessions using the database.


In [ ]:
song_playtable_create = ("""CREATE TABLE IF NOT EXISTS Song_spotify(
  Energy VARCHAR,
  Artist VARCHAR,
  Danceability VARCHAR,
  Loudness VARCHAR,
  Speechiness VARCHAR,
  Acousticness VARCHAR,
  Instrumentalness VARCHAR
)""")

In [ ]:
cur.execute(song_playtable_create)
conn.commit()

In [ ]:
trend_song =  ("""CREATE TABLE IF NOT EXISTS Trendy_song(
  Album VARCHAR,
  Artist VARCHAR ,
  Track VARCHAR,
  Album_type VARCHAR,
  Views VARCHAR,
  Likes VARCHAR,
  Comments VARCHAR
)""")

In [ ]:
cur.execute(trend_song)
conn.commit()

In [ ]:
song_spotify_table = ("""INSERT INTO Song_spotify(
  Energy ,
  Artist ,
  Danceability ,
  Loudness ,
  Speechiness ,
  Acousticness ,
  Instrumentalness)
  VALUES (%s, %s, %s, %s, %s, %s, %s)
""")


In [ ]:
for i, row in S1_Spotify.iterrows():
    r = list(row)[:7]
    cur.execute(song_spotify_table,r)


In [565]:
Song_spotify.head()

NameError: name 'Song_spotify' is not defined

In [566]:
for i, row in S2_Spotify.iterrows():
   # r = list(row)[:7]
    cur.execute(trend_song,r)


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
